In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None);
pd.set_option('display.max_rows', None);
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_colwidth', None)

In [ ]:
df=pd.read_csv('../input/home-credit-default-risk/installments_payments.csv')

In [ ]:
df.SK_ID_CURR.nunique()

In [ ]:
#339583 rows × 89 columns

In [ ]:
df.head()

In [ ]:
df["NEW_DELAY"] = df["DAYS_INSTALMENT"] - df["DAYS_ENTRY_PAYMENT"]

In [ ]:
def installments(num_rows=None):
    pd.options.mode.chained_assignment = None
    df = pd.read_csv("../input/home-credit-default-risk/installments_payments.csv")
    df["NEW_DELAY"] = df["DAYS_INSTALMENT"] - df["DAYS_ENTRY_PAYMENT"] 
    df["NEW_PAYMENT_DIFF"] = df["AMT_INSTALMENT"] - df["AMT_PAYMENT"]
    
    df["NUM_INSTALMENT_VERSION"] = df["NUM_INSTALMENT_VERSION"].astype("object")
    df[(df["NUM_INSTALMENT_VERSION"] != 1) & (df["NUM_INSTALMENT_VERSION"] != 0) & (df["NUM_INSTALMENT_VERSION"] != 2) 
       & (df["NUM_INSTALMENT_VERSION"] != 3)]['NUM_INSTALMENT_VERSION'] = 4
    
    cat_features = list(df.select_dtypes(['object']).columns)
    df = pd.get_dummies(df, columns= cat_features,drop_first=True)
    
    
    agg1 = {'SK_ID_CURR': ['count','max'],#how many monts does a customer paid.
           'NEW_DELAY': ['max', 'min', 'mean','std', 'sum'],
           'NUM_INSTALMENT_NUMBER':['min','max'], #her bir eski kredi için min ve max değerleri
           'DAYS_INSTALMENT':['max','min','std'], # bu aggden sonra belli bir sürenin üstündekileri 1 ve 0 olarak 2 ye ayırabiliriz.
           'NEW_PAYMENT_DIFF': ['max', 'mean', 'std', 'min','sum'],
           'AMT_INSTALMENT': ['max', 'mean', 'sum', 'min', 'std'],
           'AMT_PAYMENT': ['min', 'max', 'mean', 'sum', 'std'],
           'DAYS_ENTRY_PAYMENT': ['max', 'min', 'std'],
           "NUM_INSTALMENT_VERSION_1.0":["sum"],
           "NUM_INSTALMENT_VERSION_2.0":["sum"],
           "NUM_INSTALMENT_VERSION_3.0":["sum"],
           "NUM_INSTALMENT_VERSION_4.0":["sum"]
            }
    Installments_agg = df.groupby(['SK_ID_PREV']).agg(agg1)
    Installments_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in Installments_agg.columns.tolist()])
    
    Installments_agg['NEW_DAYS_INSTALMENT_NUMBER']=Installments_agg['DAYS_INSTALMENT_MAX']-Installments_agg['DAYS_INSTALMENT_MIN'] #toplam kaç günlük ödemesi var bilgisi. 
    Installments_agg['NEW_AMT_INSTALMENT_DIFF']=Installments_agg['AMT_INSTALMENT_MAX']-Installments_agg['AMT_INSTALMENT_MIN']
    agg2= {'SK_ID_CURR_COUNT':['min', 'max'],#Farklı kredilerde kaçar ay ödeme yapılmış.
           'NEW_DELAY_MAX':['min', 'max', 'mean'],#Farklı kredilerde maximum geciktirme miktarının minimumu maximumu ve ortalaması.
           'NEW_DELAY_MIN':['min', 'max', 'mean'],# aynısının min için olanı
           'NEW_DELAY_MEAN':['min', 'max', 'mean'],# xd
           'NEW_DELAY_STD':['min', 'max', 'mean'],
           'NEW_DELAY_SUM':['min', 'max', 'mean', 'sum', 'std'],
           'NUM_INSTALMENT_NUMBER_MIN':['min','max','mean'], # 1 olanlar bitmiş krediler YENİ DEĞİŞKEN OLUŞTUR.
           'NUM_INSTALMENT_NUMBER_MAX':['min','max','mean','sum'],#farklı krediler için maksimum taksit sayısı. sum=toplam kaç aylık kredi almış.
           'NEW_DAYS_INSTALMENT_NUMBER':['min','max','std'], #farklı krediler için kaç günlük krediler ödenmiş bilgisi.
           'DAYS_INSTALMENT_STD':['min','max','std'], # burdan sonrası için düşünemedim beynim çalışmıyor.
           'DAYS_INSTALMENT_MIN':['std','min','max'],
           'DAYS_INSTALMENT_MAX':['std','min','max'],
           'NEW_PAYMENT_DIFF_MAX':['min', 'max', 'mean'],
           'NEW_PAYMENT_DIFF_MEAN':['min', 'max', 'mean'],
           'NEW_PAYMENT_DIFF_SUM':['min', 'max', 'mean'],
           'NEW_PAYMENT_DIFF_STD':['min', 'max', 'mean'],
           'NEW_PAYMENT_DIFF_MIN':['min', 'max', 'mean'],
           'AMT_INSTALMENT_MAX':['min', 'max', 'mean'],
           'AMT_INSTALMENT_MEAN':['min', 'max', 'mean'],
           'AMT_INSTALMENT_SUM':['min', 'max', 'mean'],
           'AMT_INSTALMENT_STD':['min', 'max', 'mean'],
           'AMT_INSTALMENT_MIN':['min', 'max', 'mean'],
           'NEW_AMT_INSTALMENT_DIFF':['min','max','mean'],
           'AMT_PAYMENT_MIN':['min', 'max', 'mean'],
           'AMT_PAYMENT_MAX':['min', 'max', 'mean'],
           'AMT_PAYMENT_MEAN':['min', 'max', 'mean'],
           'AMT_PAYMENT_STD':['min', 'max', 'mean'],
           'AMT_PAYMENT_SUM':['min', 'max', 'mean'],
           'DAYS_ENTRY_PAYMENT_MIN':['min', 'max', 'mean'],
           'DAYS_ENTRY_PAYMENT_STD':['min', 'max', 'mean'],
           'DAYS_ENTRY_PAYMENT_MAX':['min', 'max', 'mean'],
           'NUM_INSTALMENT_VERSION_1.0_SUM':['sum','min','max'],
           'NUM_INSTALMENT_VERSION_2.0_SUM':['sum','min','max'],
           'NUM_INSTALMENT_VERSION_3.0_SUM':['sum','min','max'],
           'NUM_INSTALMENT_VERSION_4.0_SUM':['sum','min','max']
    }
    Installments_agg2=Installments_agg.groupby('SK_ID_CURR_MAX').agg(agg2)
    Installments_agg2.rename(columns={'SK_ID_CURR_MAX': 'SK_ID_CURR'},inplace=True)
    Installments_agg2.columns = pd.Index("INSTAL_" + e[0] + "_" + e[1].upper() for e in Installments_agg2.columns.tolist())
    return(Installments_agg2)

In [ ]:
df=installments()

In [ ]:
df.head()

In [ ]:
a=df.groupby('SK_ID_CURR').NEW_DELAY.apply(lambda x: pd.Series([(x < 0).sum(), (x >= 0).sum()])).unstack()
#df['LATER'] = df['NEW_DELAY'].map(lambda x: 1 if x<0 else 0)
# SK_ID_PREV in agg alırsak önce bunu Curr ü agglerken kullanabiliriz. Kaç defa geciktirmiş bilgisi.

In [ ]:
b=a.reset_index(col_level=None)

In [ ]:
a.drop('SK_ID_CURR',axis=1,inplace=True)

In [ ]:
b=pd.concat([df,a],axis=1)

In [ ]:
b.tail(50)

In [ ]:
a.head()

In [ ]:
c.isnull().sum()

In [ ]:
e=d.rename(columns={'0':'NEW_PAIDONTIME','1':'NEW_DIDNT_PAIDONTIME'})

In [ ]:
e.head()

In [ ]:
df.head()

In [ ]:
a.head()

In [ ]:
     df= pd.read_csv('../input/home-credit-default-risk/previous_application.csv')

In [ ]:
df.head()

In [ ]:
df.NAME_PORTFOLIO.value_counts()

In [ ]:
df=pd.read_csv('../input/home-credit-default-risk/previous_application.csv')

In [ ]:
df.SK_ID_PREV.value_counts()

In [ ]:
df.shape

In [ ]:
def dfPrevApp():
    dfPrevApp = pd.read_csv("../input/home-credit-default-risk/previous_application.csv")
    
    dfPrevApp.replace(365243,np.nan,inplace = True)
    dfPrevApp.replace("XNA",np.nan,inplace = True)#

    dfPrevApp['NEW_RETURN_DAY'] = dfPrevApp['DAYS_DECISION'] + dfPrevApp['CNT_PAYMENT'] * 30

    dfPrevApp['NEW_DAYS_TERMINATION_diff'] = dfPrevApp['DAYS_TERMINATION'] - dfPrevApp['NEW_RETURN_DAY']

    dfPrevApp['NEW_AMT_DOWN_PAYMENT_rate'] = dfPrevApp['AMT_DOWN_PAYMENT'] / (dfPrevApp['AMT_CREDIT'] + 0.01)

    dfPrevApp['NEW_AMT_SPEND_TO_PRODUCT'] = dfPrevApp['AMT_GOODS_PRICE'] / dfPrevApp['AMT_CREDIT'] # Alınan kredinin ürün alışverişine harcanma oranı
    
    dfPrevApp['NEW_DAYS_DUE']=dfPrevApp['DAYS_FIRST_DUE'] - dfPrevApp['DAYS_LAST_DUE_1ST_VERSION'] 
    
    dfPrevApp['NEW_APP_CREDIT_PERC'] = dfPrevApp['AMT_APPLICATION'] / dfPrevApp['AMT_CREDIT']
    


    
    dfPrevApp["NAME_PAYMENT_TYPE"].replace(["Non-cash from your account","Cashless from the account of the employer"],np.nan,inplace=True)

    a = ["Channel of corporate sales","Car dealer"]
    dfPrevApp["CHANNEL_TYPE"].replace(a,"Others_Type",inplace=True)

    b = ['Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people'] 
    dfPrevApp["NAME_TYPE_SUITE"] = dfPrevApp["NAME_TYPE_SUITE"].replace(b, 'not_alone')


    dfPrevApp["WEEKDAY_APPR_PROCESS_START"] = dfPrevApp["WEEKDAY_APPR_PROCESS_START"].replace(['MONDAY','TUESDAY', 'WEDNESDAY','THURSDAY','FRIDAY'], 'WEEK_DAY') 
    dfPrevApp["WEEKDAY_APPR_PROCESS_START"] = dfPrevApp["WEEKDAY_APPR_PROCESS_START"].replace(['SATURDAY', 'SUNDAY'], 'WEEKEND')

    a = ['Auto technology', 'Jewelry', 'MLM partners', 'Tourism'] 
    dfPrevApp["NAME_SELLER_INDUSTRY"] = dfPrevApp["NAME_SELLER_INDUSTRY"].replace(a, 'Other_Ind')

    a = ['Auto Accessories', 'Jewelry', 'Homewares', 'Medical Supplies', 'Vehicles', 'Sport and Leisure','Gardening', 'Other', 'Office Appliances', 'Tourism', 'Medicine', 'Direct Sales', 'Fitness', 'Additional Service','Education', 'Weapon', 'Insurance', 'House Construction', 'Animals']  
    dfPrevApp["NAME_GOODS_CATEGORY"] = dfPrevApp["NAME_GOODS_CATEGORY"].replace(a, 'Other_Cat')

    a = ['Buying a used car','Building a house or an annex','Everyday expenses','Medicine','Payments on other loans','Education','Journey', 'Purchase of electronic equipment','Buying a new car','Wedding / gift / holiday','Buying a home','Car repairs','Furniture','Buying a holiday home / land', 'Business development','Gasification / water supply','Buying a garage','Hobby','Money for a third person','Refusal to name the goal','Urgent needs','Other']
    dfPrevApp['NAME_CASH_LOAN_PURPOSE']= dfPrevApp['NAME_CASH_LOAN_PURPOSE'].replace(a,'Others')

    dfPrevApp["NAME_PORTFOLIO"].replace("cars",np.nan,inplace=True)
    
    a = [8,9,10,11,12,13,14,15,16,17]
    dfPrevApp["HOUR_APPR_PROCESS_START"] = dfPrevApp["HOUR_APPR_PROCESS_START"].replace(a, 'Working_Hours')

    b = [18,19,20,21,22,23,0,1,2,3,4,5,6,7]
    dfPrevApp["HOUR_APPR_PROCESS_START"] = dfPrevApp["HOUR_APPR_PROCESS_START"].replace(b, 'Off_Hours')
    #------------------------------------------
    drops = ["RATE_INTEREST_PRIMARY","RATE_INTEREST_PRIVILEGED","FLAG_LAST_APPL_PER_CONTRACT","NFLAG_LAST_APPL_IN_DAY","NAME_PRODUCT_TYPE","SELLERPLACE_AREA"]
    dfPrevApp.drop(drops,inplace=True,axis=1)
    
    dfPrevApp["NFLAG_INSURED_ON_APPROVAL"] = dfPrevApp["NFLAG_INSURED_ON_APPROVAL"].astype("object")
    cat_features = list(dfPrevApp.select_dtypes(['object']).columns)
    dfPrevApp = pd.get_dummies(dfPrevApp, columns= cat_features, dummy_na= True,drop_first=True)
    
    agg1 = {'SK_ID_CURR': ['size'],# kaç tane kredi almış önceden
    'AMT_ANNUITY': ['max', 'min', 'mean','std', 'sum'], 
    'AMT_APPLICATION':['max', 'min', 'mean','std', 'sum'],
    'AMT_CREDIT':['max', 'min', 'mean','std', 'sum'],
    'AMT_DOWN_PAYMENT': ['max', 'min', 'mean','std', 'sum'],
    'AMT_GOODS_PRICE': ['max', 'min', 'mean','std', 'sum'],
    'RATE_DOWN_PAYMENT': ['max', 'min', 'mean','std'],
    'DAYS_DECISION': ['max', 'min', 'mean', 'sum'],
    'CNT_PAYMENT': ['max', 'min', 'mean','std', 'sum'],
    'DAYS_FIRST_DRAWING': ['max', 'min', 'mean', 'sum'],
    'DAYS_FIRST_DUE': ['max', 'min', 'mean', 'sum'],
    'DAYS_LAST_DUE_1ST_VERSION': ['max', 'min', 'mean', 'sum'],
    'DAYS_LAST_DUE': ['max', 'min', 'mean', 'sum'],
    'DAYS_TERMINATION': ['max', 'min', 'mean','std', 'sum'],#DAYS_TERMINATION-DAYS_LAST_DUE
    'NEW_RETURN_DAY': ['max', 'min', 'mean','std', 'sum'],
    'NEW_DAYS_TERMINATION_diff': ['max', 'min', 'mean','std', 'sum'],
    'NEW_AMT_DOWN_PAYMENT_rate': ['max', 'min', 'mean','std'],
    'NEW_AMT_SPEND_TO_PRODUCT': ['max', 'min', 'mean','std', 'sum'],
    'NEW_APP_CREDIT_PERC': ['max', 'min', 'mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['max', 'min','sum'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['max', 'min','sum'],
    'NAME_CONTRACT_TYPE_nan': ['max', 'min','sum'],
    'WEEKDAY_APPR_PROCESS_START_WEEK_DAY': ['max', 'min', 'sum'],
    'WEEKDAY_APPR_PROCESS_START_nan': ['max', 'min', 'sum'],
    'HOUR_APPR_PROCESS_START_Working_Hours': ['max', 'min', 'sum'],
    'HOUR_APPR_PROCESS_START_nan': ['max', 'min', 'sum'],
    'NAME_CASH_LOAN_PURPOSE_Repairs': ['max', 'min', 'sum'],
    'NAME_CASH_LOAN_PURPOSE_XAP': ['max', 'min', 'sum'],
    'NAME_CASH_LOAN_PURPOSE_nan': ['max', 'min',  'sum'],
    'NAME_CONTRACT_STATUS_Canceled': ['max', 'min', 'sum'],
    'NAME_CONTRACT_STATUS_Refused': ['max', 'min', 'sum'],
    'NAME_CONTRACT_STATUS_Unused offer': ['max', 'min', 'sum'],
    'NAME_CONTRACT_STATUS_nan': ['max', 'min', 'sum'],
    'NAME_PAYMENT_TYPE_nan': ['max', 'min', 'sum'],
    'CODE_REJECT_REASON_HC': ['max', 'min','sum'],
    'CODE_REJECT_REASON_LIMIT': ['max', 'min','sum'],
    'CODE_REJECT_REASON_SCO': ['max', 'min','sum'],
    'CODE_REJECT_REASON_SCOFR': ['max', 'min', 'sum'],
    'CODE_REJECT_REASON_SYSTEM': ['max', 'min', 'sum'],
    'CODE_REJECT_REASON_VERIF': ['max', 'min', 'sum'],
    'CODE_REJECT_REASON_XAP': ['max', 'min', 'sum'],
    'CODE_REJECT_REASON_nan': ['max', 'min','sum'],
    'NAME_TYPE_SUITE_not_alone': ['max', 'min','sum'],
    'NAME_TYPE_SUITE_nan': ['max', 'min', 'sum'],
    'NAME_CLIENT_TYPE_Refreshed': ['max', 'min','sum'],
    'NAME_CLIENT_TYPE_Repeater': ['max', 'min', 'sum'],
    'NAME_CLIENT_TYPE_nan': ['max', 'min','sum'],
    'NAME_GOODS_CATEGORY_Clothing and Accessories': ['max', 'min', 'sum'],
    'NAME_GOODS_CATEGORY_Computers': ['max', 'min','sum'],
    'NAME_GOODS_CATEGORY_Construction Materials': ['max', 'min', 'sum'],
    'NAME_GOODS_CATEGORY_Consumer Electronics': ['max', 'min', 'sum'],
    'NAME_GOODS_CATEGORY_Furniture': ['max', 'min', 'sum'],
    'NAME_GOODS_CATEGORY_Mobile': ['max', 'min', 'sum'],
    'NAME_GOODS_CATEGORY_Other_Cat': ['max', 'min', 'sum'],
    'NAME_GOODS_CATEGORY_Photo / Cinema Equipment': ['max', 'min', 'sum'],
    'NAME_GOODS_CATEGORY_nan': ['max', 'min', 'sum'],
    'NAME_PORTFOLIO_Cars': ['max', 'min', 'sum'],
    'NAME_PORTFOLIO_Cash': ['max', 'min', 'sum'],
    'NAME_PORTFOLIO_POS': ['max', 'min','sum'],
    'NAME_PORTFOLIO_nan': ['max', 'min', 'sum'],
    'CHANNEL_TYPE_Contact center': ['max', 'min', 'sum'],
    'CHANNEL_TYPE_Country-wide': ['max', 'min', 'sum'],
    'CHANNEL_TYPE_Credit and cash offices': ['max', 'min', 'sum'],
    'CHANNEL_TYPE_Others_Type': ['max', 'min', 'sum'],
    'CHANNEL_TYPE_Regional / Local': ['max', 'min','sum'],
    'CHANNEL_TYPE_Stone': ['max', 'min','sum'],
    'CHANNEL_TYPE_nan': ['max', 'min', 'sum'],
    'NAME_SELLER_INDUSTRY_Connectivity': ['max', 'min','sum'],
    'NAME_SELLER_INDUSTRY_Construction': ['max', 'min', 'sum'],
    'NAME_SELLER_INDUSTRY_Consumer electronics': ['max', 'min',  'sum'],
    'NAME_SELLER_INDUSTRY_Furniture': ['max', 'min', 'sum'],
    'NAME_SELLER_INDUSTRY_Industry': ['max', 'min', 'sum'],
    'NAME_SELLER_INDUSTRY_Other_Ind': ['max', 'min','sum'],
    'NAME_SELLER_INDUSTRY_nan': ['max', 'min','sum'],
    'NAME_YIELD_GROUP_low_action': ['max', 'min', 'sum'],
    'NAME_YIELD_GROUP_low_normal': ['max', 'min', 'sum'],
    'NAME_YIELD_GROUP_middle': ['max', 'min','sum'],
    'NAME_YIELD_GROUP_nan': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_Card X-Sell': ['max', 'min', 'sum'],
    'PRODUCT_COMBINATION_Cash': ['max', 'min', 'sum'],
    'PRODUCT_COMBINATION_Cash Street: high': ['max', 'min', 'sum'],
    'PRODUCT_COMBINATION_Cash Street: low': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_Cash Street: middle': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_Cash X-Sell: high': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_Cash X-Sell: low': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_Cash X-Sell: middle': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS household with interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS household without interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS industry with interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS industry without interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS mobile with interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS mobile without interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS other with interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_POS others without interest': ['max', 'min','sum'],
    'PRODUCT_COMBINATION_nan': ['max', 'min','sum'],
    'NFLAG_INSURED_ON_APPROVAL_1.0': ['max', 'min','sum'],
    'NFLAG_INSURED_ON_APPROVAL_nan': ['max', 'min','sum']
}
    
   
    
    dfPrevApp = dfPrevApp.groupby(['SK_ID_CURR']).agg(agg1)
    
    dfPrevApp.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in dfPrevApp.columns.tolist()])

    return (dfPrevApp)

In [ ]:
dfPrevApp_new=dfPrevApp()

In [ ]:
dfPrevApp_new.head()